In [ ]:
!nvidia-smi

Thu Feb 29 17:52:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 460 (delta 131), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (460/460), 126.19 KiB | 4.51 MiB/s, done.
Resolving deltas: 100% (233/233), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.2 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has a Tesla T4 GPU!
We will install the latest stable RAPIDS via pip 24.2.*!  Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.7/464.7 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.3 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import numpy as np
import pandas as pd

import cupy as cp
import cudf
from cuml import LinearRegression
from cuml.preprocessing import MinMaxScaler
from tqdm import tqdm

## Preprocess data

In [ ]:
!wget https://github.com/amanlai/datasets/raw/main/store-sales-time-series-forecasting.zip

--2024-02-29 19:00:41--  https://github.com/amanlai/datasets/raw/main/store-sales-time-series-forecasting.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/amanlai/datasets/main/store-sales-time-series-forecasting.zip [following]
--2024-02-29 19:00:41--  https://raw.githubusercontent.com/amanlai/datasets/main/store-sales-time-series-forecasting.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22416355 (21M) [application/zip]
Saving to: ‘store-sales-time-series-forecasting.zip’

store-sales-time-se 100%[===================>]  21.38M  --.-KB/s    in 0.1s    

2024-02-29 19:00:42 (208 MB/s

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
def read_data(filename, path='/content/store-sales-time-series-forecasting.zip', dateformat=None, **kwargs):
    with zipfile.ZipFile(path) as zip_file:
        with zip_file.open(filename) as f:
            df = cudf.read_csv(f, **kwargs)
            if dateformat is not None:
                df['date'] = cudf.to_datetime(df['date'], format=dateformat)
            return df



def reshape_df(df, index='date', columns='store_nbr', values=None, end=None):

    p = df.pivot(index=index, columns=columns, values=values)
    if end is not None:
        p = p[p.index < end]
    p = p.interpolate(method='linear').bfill()
    return p



def transform_df(df, columns=None, log_transform=False, is_train=False):

    index = df.index
    df_array = df.values

    if log_transform:
        df_array = cp.log1p(df_array)

    sc = MinMaxScaler()
    scaled = sc.fit_transform(df_array)
    transformed = cudf.DataFrame(scaled, index=index, columns=columns)

    if is_train:
        return transformed, sc
    else:
        return transformed



def get_target_data(df, target_cols='sales', train_end=None):

    p = reshape_df(df, 'date', 'store_nbr', target_cols, train_end)
    columns = p.columns.rename('store_nbr')
    result, sc = transform_df(p, columns, True, True)

    # create the static variables
    xy = df[['store_nbr', 'city', 'state', 'type', 'cluster']].drop_duplicates()
    dummies = cudf.get_dummies(xy, columns=xy.columns, dtype='int32')
    cols = dummies.columns.tolist()
    dummies = dummies.sort_values(cols, ascending=False, ignore_index=True)
    # since `dummies` is sorted, it's safe to assign `store_nbr` as its values
    dummies.index = result.columns
    return result, sc, dummies



def get_past_covariates(df, past_cols=None, train_end=None):

    p = reshape_df(df, 'date', 'store_nbr', past_cols, train_end)
    columns = cudf.MultiIndex.from_tuples(p.columns.tolist(), names=['past', 'store_nbr'])
    result = transform_df(p, columns)
    return result


def get_future_covariates(
    df,
    future_cols=None,
    future_ma_cols=None,
    future_window_sizes=None
):

    p = reshape_df(df, 'date', 'store_nbr', future_cols)
    columns = cudf.MultiIndex.from_tuples(p.columns.tolist(), names=['future', 'store_nbr'])
    transformed = transform_df(p, columns)
    ma_dfs = []
    for window_size in future_window_sizes:
        cols = cudf.MultiIndex.from_product([
            [f'{c}_ma{window_size}' for c in future_ma_cols],
            transformed.columns.levels[1]
        ], names=['future', 'store_nbr'])
        # we need a left closed window
        # (which means we need shifting the default right closed solution)
        shift = -window_size//2 + 1
        ma = transformed[future_ma_cols].rolling(f'{window_size}D').mean().shift(shift)
        # we need to center the window as well
        # adjustment for centering
        idx = transformed.index[-window_size+1 : -window_size+1-shift].to_arrow().tolist()
        new_values = [transformed.loc[i:, future_ma_cols].values.mean(axis=0) for i in idx]
        ma.iloc[shift:] = new_values
        ma.columns = cols
        ma_dfs.append(ma)
    result = cudf.concat([transformed, *ma_dfs], axis=1)
    return result




def shift_df(df, i):

    result = df.shift(i)
    if isinstance(df.columns, pd.MultiIndex):
        result.columns = cudf.MultiIndex.from_tuples([(i, feature, store) for (feature, store) in result.columns], names=['i', *df.columns.names])
    else:
        result.columns = cudf.MultiIndex.from_tuples([(i, '0', store) for store in result.columns], names=['i', 'lag', df.columns.name])
    return result



def get_training_data(train_df, past_covariates, future_covariates, dummy_vars):

    # shift the variables for appropriate number of time delta
    tmp = [
        *(shift_df(train_df, i) for i in lags['target']),
        *(shift_df(past_covariates, i) for i in lags['past']),
        *(shift_df(future_covariates, i) for i in lags['future'])
    ]

    X_train = cudf.concat(tmp, axis=1)                  # concatenate the variables horizontally

    old_column_order = (
        X_train.columns
        .droplevel(-1)        # remove the innermost level because we want to find the unique lag-feature pairs
        .drop_duplicates()    # get unique pairs
    )

    X_train = (
        X_train
        .stack(level=-1)
        [old_column_order]                              # preserve old column order
    )

    # flatten column labels
    X_train.columns = X_train.columns.map("{0[1]}_{0[0]}".format)
    X_train.index.names = ['date', 'store_nbr']

    X_train = (
        X_train
        .dropna()                                       # drop NaNs created by shift
        .reset_index()                                  # recover `date`, `store_nbr` as columns
        .merge(dummy_vars, on='store_nbr', how='left')  # merge the static variables on `store_nbr`
        .set_index(['date', 'store_nbr'])               # make `date`, `store_nbr` back into index
        .sort_index(level=['store_nbr', 'date'])        # sort by the store_nbr
    )

    y_train = train_df.stack().loc[X_train.index]         # align y_train with X_train

    return X_train, y_train



def predict(
    model,
    n,
    train_df,
    past_covariates,
    future_covariates,
    dummy_vars,
    scaler,
    zero_forecast,
):

    relative_cov_lags = {
        'past': (cp.array(lags['past']) - min(lags['past']))[::-1],
        'future': (cp.array(lags['future']) - min(lags['future']))[::-1],
    }

    idx = train_df.columns
    target_array = train_df.tail(max(lags['target'])).values    # use the underlying array

    past_covs_array = past_covariates.tail(max(lags['past']))
    # we can safely drop the outer level because past covariates consist of only one variable: transactions.
    past_covs_array.columns = past_covs_array.columns.droplevel(0)
    # sort so that the store numbers match
    # use the underlying array
    past_covs_array = past_covs_array[idx].values

    dummy_vars = dummy_vars.loc[idx]   # sort so that the store numbers match

    cutoff = train_df.index.max() - pd.Timedelta(days=max(lags['future'])-1)
    cov_df = future_covariates.loc[cutoff:]

    # this is useful to construct a MultiIndex to sort the frame to match X_train order
    feature_labels = cov_df.columns.get_level_values(0).drop_duplicates().tolist()

    # prediction
    predictions = []
    # t_pred indicates the number of time steps after the first prediction
    for t_pred in range(n):
        # concatenate the previous day's predictions to the target
        if predictions:
            last_prediction = predictions[-1][None, :]
            target_array = cp.concatenate((target_array, last_prediction), axis=0)
        # prepare the target variable for prediction
        tmp_X = target_array[-cp.array(lags['target'])]
        # prepare past covariates for prediction
        past_cov = past_covs_array[relative_cov_lags["past"] + t_pred]


        # prepare future covariates for prediction
        # select relevant dates for prediction
        future_cov = cov_df.iloc[relative_cov_lags['future'] + t_pred]

        index_order = cudf.MultiIndex.from_product([
            future_cov.index.to_arrow().tolist(),
            feature_labels       # the features level
        ], names=['date', 'future'])

        future_cov = (
            future_cov
            .stack(level=0)      # convert the outer column level (variable types) into index level
            .loc[index_order]    # sort the index to match X_train order
            .values              # use the underlying array
        )
        # concatenate target, past and future covariates to pass to predict()
        # finally, we need to transpose this array because the model expects stores as index
        X = cp.concatenate([tmp_X, past_cov, future_cov, dummy_vars.values.T], axis=0).T

        # prediction for a specific day
        predictions.append(model.predict(X))

    # invert the transformation
    pred_array = cp.vstack(predictions)
    inv_trans_preds = cp.expm1(scaler.inverse_transform(pred_array))
    predictions = cudf.DataFrame(
        inv_trans_preds,
        index=cudf.date_range(train_df.index.max()+pd.Timedelta(days=1), periods=n, freq='D'),
        #                                          ^^^^^^^ <--- start from next day
        columns=idx
    )

    # if the past `zero_forecast` days were 0, predict 0
    zero_mask = train_df.tail(zero_forecast).sum() == 0
    predictions.loc[:, zero_mask.values.get()] = 0
    # coerce negative predictions to be 0
    predictions = predictions.clip(0)
    return predictions

time: 10.7 ms (started: 2024-02-29 19:00:57 +00:00)


In [ ]:
# to run the JIT compiler once
x = cudf.DataFrame([[1]]).interpolate().bfill()
del x

time: 1min 17s (started: 2024-02-29 19:00:58 +00:00)


In [ ]:
train = read_data("train.csv", dateformat='%Y-%m-%d')
test = read_data("test.csv", dateformat='%Y-%m-%d')

oil = read_data('oil.csv', dateformat='%Y-%m-%d').rename(columns={"dcoilwtico": "oil"})
store = read_data("stores.csv")
transactions = read_data("transactions.csv", dateformat='%Y-%m-%d')
holiday_events = read_data("holidays_events.csv", dateformat='%Y-%m-%d')

time: 1.92 s (started: 2024-02-29 19:02:16 +00:00)


In [ ]:
holiday_events['description'] = (
    holiday_events
    [['description', 'locale_name']]
    .apply(lambda x: x['description'].replace(x['locale_name'], ''), axis=1)
    .str.lower()
    .str.replace(r'\-|\+|\d+|\b(de|del|traslado|recupero|puente|-)\b', '', regex=True)
    .pipe(lambda x: x.str.extract('(futbol)', expand=False).fillna(x))
    .str.replace('|'.join(store[['city', 'state']].stack().str.lower().unique().to_arrow().tolist()), '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)
holiday_events = holiday_events[~holiday_events['transferred']]

time: 2.94 s (started: 2024-02-29 19:02:18 +00:00)


Saturdays are designated as work days.

In [ ]:
work_days = (
    holiday_events
    .loc[holiday_events['type'] == 'Work Day', ["date", "type"]]
    .rename(columns={"type": "work_day"})
    .reset_index(drop=True)
)
work_days['work_day'] = work_days['work_day'].notna().astype('Int32')

# remove work days after extracting above
holiday_events = holiday_events[holiday_events['type'] != 'Work Day'].reset_index(drop=True)

time: 57.7 ms (started: 2024-02-29 19:02:21 +00:00)


In [ ]:
relevant_days = (
    holiday_events
    .loc[holiday_events['locale'] == 'National', ["date", "description"]]
    .drop_duplicates()
)

dummified = cudf.get_dummies(relevant_days, columns=['description'], prefix='holiday', dtype='int32')

# some dates have multiple holidays, so put them all in a single row
national_holidays = dummified.groupby('date', as_index=False).sum()

# not all holidays are impactful
# keep some national holidays with larger impacts on sales
relevant_holidays = [
    'holiday_dia difuntos', 'holiday_dia la madre',
    'holiday_dia trabajo', 'holiday_futbol', 'holiday_navidad',
    'holiday_primer dia ano', 'holiday_terremoto'
]
national_holidays = national_holidays[['date', *relevant_holidays]]

time: 136 ms (started: 2024-02-29 19:02:21 +00:00)


In [ ]:
train_start = train['date'].min()
test_start = test['date'].min()
train_end = train['date'].max() + pd.Timedelta(days=1)   # have to add 1D more because cudf date_range doesn't include right
test_end = test['date'].max() + pd.Timedelta(days=1)

time: 52.2 ms (started: 2024-02-29 19:02:21 +00:00)


In [ ]:
# reindex training data
multi_idx = cudf.MultiIndex.from_product(
    [
        cudf.date_range(train_start, train_end, freq='D').to_arrow(),
        train['store_nbr'].unique().to_arrow(),
        train['family'].unique().to_arrow()
    ],
    names=['date', 'store_nbr', 'family'],
)
# this adds missing dates
# however this generates missing sales, id and on-promotion values
train = train.set_index(['date', 'store_nbr', 'family']).reindex(multi_idx).reset_index()

# fill missing values with zeros
# the assumption here is that days where there were no sales were probably not recorded
train[['sales', 'onpromotion']] = train[['sales', 'onpromotion']].fillna(0)
# interpolate linearly as a filler for the 'id'
# not really useful for training but is useful for predicting test data
train['id'] = train['id'].interpolate(method="linear")

time: 4.47 s (started: 2024-02-29 19:02:21 +00:00)


In [ ]:
# compute total sales for each store
store_sales = train.groupby(['date', 'store_nbr'], as_index=False)['sales'].sum()

# reindex transactions data
# same as the reindexing of `train`, this adds more dates and missing transactions
transactions = (
    transactions
    .merge(store_sales, on=['date', 'store_nbr'], how='outer')
    .sort_values(['date', 'store_nbr'], ignore_index=True)     # important for interpolation later on
)

# if there were zero sales, it is a good assumption that there would be zero transactions
transactions.loc[transactions['sales'] == 0, 'transactions'] = 0
transactions = transactions.drop(columns=['sales'])


# fill remaining missing values using linear interpolation
transactions['transactions'] = (
    transactions
    .groupby('store_nbr', group_keys=False)['transactions']
    .apply(lambda x: x.interpolate(method='linear').bfill())
)

time: 455 ms (started: 2024-02-29 19:02:25 +00:00)


In [ ]:
# oil data is business day time series (therefore missing weekends)
# add those dates in by reindexing date

# date index of the date range
idx = cudf.date_range(train_start, test_end, freq='D', name='date')
# add the missing dates
oil = oil.set_index('date').reindex(idx).reset_index()
# interpolate the missing prices
oil['oil'] = oil['oil'].interpolate(method='linear').bfill()

time: 215 ms (started: 2024-02-29 19:02:26 +00:00)


In [ ]:
# combine all the datasets
data = (
    cudf.concat([train, test])
    .merge(transactions, on=['date', 'store_nbr'], how='left')
    .merge(oil, on='date', how='left')
    .merge(store, on='store_nbr', how='left')
    .merge(work_days, on='date', how='left')
    .merge(national_holidays, on='date', how='left')
    .sort_values(['date', 'store_nbr', 'family'], ignore_index=True)
)

# fill columns with 0s to indicate absence of holidays/events
data[["work_day", *relevant_holidays]] = data[["work_day", *relevant_holidays]].fillna(0)

time: 298 ms (started: 2024-02-29 19:02:26 +00:00)


In [ ]:
# include date-related future covariates
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_year'] = data['date'].dt.dayofyear
data['week_of_year'] = data['date'].dt.isocalendar().week.astype('int32')
data['date_index'] = data['date'].factorize()[0]    # trend

time: 53.4 ms (started: 2024-02-29 19:02:26 +00:00)


In [ ]:
# impute days with zero sales using linear interpolation later
# there were no sales on new year's days
missing_dates = cudf.date_range(train_start, train_end, freq='D').difference(train['date'].unique())
new_years_days = cudf.date_range(train_start.astype('datetime64[Y]'), f"{test_end.year}", freq=cudf.DateOffset(years=1))
zero_sales_dates = missing_dates.union(new_years_days)

zero_sales_mask = data['date'].isin(zero_sales_dates) & (data['sales'] == 0) & (data['onpromotion'] == 0)
data.loc[zero_sales_mask, ['sales', 'onpromotion']] = cudf.NA

time: 85.5 ms (started: 2024-02-29 19:02:26 +00:00)


In [ ]:
data.shape

(3036528, 27)

time: 2.8 ms (started: 2024-02-29 19:02:27 +00:00)


## Train model

### Prepare raw training data, and past and future covariates

In [ ]:
past_cols = ["transactions"]

future_cols = [
    "oil", "onpromotion",
    "day", "month", "year", "day_of_week", "day_of_year", "week_of_year", "date_index",
    "work_day", *relevant_holidays
]
future_ma_cols = ["oil", "onpromotion"]
future_window_sizes = [7, 28]

lags = {
    "target": sorted(range(1, 64), reverse=True),
    "past": sorted(range(16, 23), reverse=True),
    "future": [-i for i in range(-14, 1)]
}

time: 1.16 ms (started: 2024-02-29 19:02:27 +00:00)


### Prepare training data

In [ ]:
n = data['family'].nunique()
predictions = []

with tqdm(total=n) as pbar:
    for family, g in data.groupby('family'):

        df, scaler, dummy = get_target_data(g, train_end=train_end)
        past = get_past_covariates(g, past_cols, train_end)
        future = get_future_covariates(g, future_cols, future_ma_cols, future_window_sizes)

        X_train, y_train = get_training_data(df, past, future, dummy)

        # fit a model
        lr = LinearRegression(algorithm='svd', copy_X=False)
        lr.fit(X_train.values, y_train.values)

        prediction = predict(lr, 16, df, past, future, dummy, scaler, zero_forecast=21)
        predictions.append(prediction.assign(family=family))
        pbar.update(1)

100%|██████████| 33/33 [14:13<00:00, 25.88s/it]


In [ ]:
final_predictions = (
    cudf.concat(predictions)
    .set_index('family', append=True)
    .stack()
)

## Predictions

In [ ]:
ids = test.set_index(['date', 'family', 'store_nbr'])[['id']]
final_predictions.index.names = ['date', 'family', 'store_nbr']
final_predictions.name = 'sales'

In [ ]:
df = ids.join(final_predictions).reset_index(drop=True).sort_values(by='id')

In [ ]:
df.to_csv('submission.csv', index=False)

## Scratch

## Benchmark

In [ ]:
aa = 'GROCERY I'
bb = 'BEVERAGES'
g = data[data['family']==bb]

time: 16.8 ms (started: 2024-02-29 19:18:27 +00:00)


In [ ]:
df, scaler, dummy = get_target_data(g, train_end=train_end)

time: 9.31 s (started: 2024-02-29 19:18:31 +00:00)


In [ ]:
past = get_past_covariates(g, past_cols, train_end)

time: 458 ms (started: 2024-02-29 19:18:40 +00:00)


In [ ]:
future = get_future_covariates(g, future_cols, future_ma_cols, future_window_sizes)

time: 7.17 s (started: 2024-02-29 19:18:41 +00:00)


In [ ]:
X_train, y_train = get_training_data(df, past, future, dummy)

time: 22 s (started: 2024-02-29 19:18:48 +00:00)


In [ ]:
algorithm = "svd-qr"        # <---- slightly different but close to sklearn
algorithm = "svd-jacobi"    # <---- same as sklearn
# algorithm = "eig"           # <--- way off
lr = LinearRegression(algorithm=algorithm, copy_X=False)
lr.fit(X_train.values, y_train.values)

LinearRegression()